In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_processing import get_matrices, apply_kmeans_labels
import torch
import torch.nn as nn

In [5]:
from data_processing import filter_columns

In [6]:
data = pd.read_csv('../data/training.csv')

In [7]:
X, y = get_matrices(data)

/Users/prasunbanerjee/Desktop/UniversityMaterials/Semester4/CS 4641/LiquidityShock/data_processing.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('row_id', axis=1, inplace=True)
/Users/prasunbanerjee/Desktop/UniversityMaterials/Semester4/CS 4641/LiquidityShock/data_processing.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('security_id', axis=1, inplace=True)
/Users/prasunbanerjee/Desktop/UniversityMaterials/Semester4/CS 4641/LiquidityShock/data_processing.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

In [8]:
clusters = apply_kmeans_labels(X, y)

In [9]:
# naive model
from models import NaiveRepeaterModel

X_np = X.values
naive_model = NaiveRepeaterModel(output_length=20)
naive_model.fit(X_np)
preds_naive = naive_model.predict(X_np)

In [10]:
clusters.shape

(19520,)

In [11]:
# Multi-output regression model 
# We need to train five separate linear regressions
from models import MultiOutputRegression
from torch import optim

X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)
cluster_tensor = torch.tensor(clusters, dtype=torch.int64)

cluster_models = {}
num_clusters = cluster_tensor.max().item() + 1

for cluster_id in range(num_clusters):
    idx = (cluster_tensor == cluster_id)
    X_sub = X_tensor[idx]
    y_sub = y_tensor[idx]

    reg_model = MultiOutputRegression(input_dim=104, output_dim=20)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(reg_model.parameters(), lr=0.01)

    for epoch in range(100): 
        reg_model.train()
        preds = reg_model(X_sub)
        loss = criterion(preds, y_sub)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    cluster_models[cluster_id] = reg_model

In [12]:
# Train the Gaussian-Discriminant Analysis
from models import GaussianDiscriminantAnalysis

gda_model = GaussianDiscriminantAnalysis()
gda_model.fit(X_tensor, cluster_tensor)

In [13]:
# Train the SVM classifier model 
from models import SVMClassifier

svm_model = SVMClassifier()
svm_model.fit(X_np, clusters)

In [14]:
# train the transformer 
from models import TransformerRegressor

transformer_model = TransformerRegressor()
criterion = nn.MSELoss()
optimizer = optim.Adam(transformer_model.parameters(), lr=0.001)

epochs = 100

for epoch in range(epochs):
    transformer_model.train()
    
    preds = transformer_model(X_tensor)  # shape: (N, 20)
    loss = criterion(preds, y_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 10, Loss: 0.5091
Epoch 20, Loss: 0.2279
Epoch 30, Loss: 0.0713
Epoch 40, Loss: 0.0141
Epoch 50, Loss: 0.0042
Epoch 60, Loss: 0.0031
Epoch 70, Loss: 0.0025
Epoch 80, Loss: 0.0020
Epoch 90, Loss: 0.0020
Epoch 100, Loss: 0.0017
